In [1]:
import glob
import numpy as np
import pandas as pd
from astropy.time import Time
import cdflib
import datetime
import spacepy.datamodel as dm
from astropy.table import Table

spatial_window = 5
temporal_window = 15


radar_list = glob.glob('../data/external/Madrigal/2018-2021/*/*.hdf5')
radar_list.sort()
# print(len(radar_list))


gf_list = glob.glob('../data/interim/GRACEFO/KBRNE_relative_v1/dat/*')
gf_list.sort()
# print(len(gf_list))
headerlist=['CDF Epoch', 'GPS', 'Latitude', 'Longitude', 'Radius', 'Latitude_QD', 'Longitude_QD','MLT GRACE_1_Position', 'GRACE_2_Position', 'Iono_Corr', 'Distance', 'Relative_Hor_TEC', 'Relative_Ne']

gf_list_datetimes =[]
for gf_file in gf_list:
    gf = pd.read_csv(gf_file, sep='\s+', header=0,index_col=False, names=headerlist)
    gf_list_datetimes.append((Time(gf['CDF Epoch'].values[-1], format='cdf_epoch').datetime).strftime('%Y-%m-%d'))

# for i in iter(range(0,len(radar_list))):
# for i in iter(range(0,500)):
i= 265
radar = (Table.read(radar_list[i], path='Data/Table Layout')).to_pandas()
radar_date = datetime.datetime(int(radar.year[0]), int(radar.month[0]), int(radar.day[0])).strftime('%Y-%m-%d')

date, idx_gf, idx_radar = np.intersect1d(gf_list_datetimes, radar_date, return_indices=True)

if len(idx_radar) > 0:
    # print(i,date,idx_gf)

    gf_file = gf_list[idx_gf[0]]
    gf = pd.read_csv(gf_file, sep='\s+', header=0, index_col=False, names=headerlist)
    gf['dates'] = (Time(gf['CDF Epoch'].values, format='cdf_epoch')).datetime

    radar_metadata = (Table.read(radar_list[i], path='Metadata/Experiment Parameters')).to_pandas()

    names = np.array([x.decode() for x in radar_metadata['name']])
    values = np.array([x.decode() for x in radar_metadata['value']])

    radar_metadata = pd.DataFrame({'name': names,
                                   'value': values})
    radar_lat = float(radar_metadata[radar_metadata['name'] == 'instrument latitude']['value'].values[0])
    radar_lon = float(radar_metadata[radar_metadata['name'] == 'instrument longitude']['value'].values[0])

    gf = gf[((gf['Latitude'] <= radar_lat + spatial_window) & (gf['Latitude'] >= radar_lat - spatial_window))]
    gf = gf[((gf['Longitude'] <= radar_lon + spatial_window) & (gf['Longitude'] >= radar_lon - spatial_window))]

    if len(gf) > 0:
        # print('------')
        # print(i, date, len(gf))
        # print('------')

        gf_pass = gf[gf.diff(axis=0)['dates'] > datetime.timedelta(minutes=temporal_window)]
        # print(len(gf_pass))

        if len(gf_pass) == 0:  # only one passage

            time_window_i = gf['dates'][gf.index.min()] - datetime.timedelta(minutes=temporal_window)
            time_window_f = gf['dates'][gf.index.max()] + datetime.timedelta(minutes=temporal_window)

            dates = []
            for d in iter(range(0, len(radar))):
                dates.append(datetime.datetime(int(radar.year[d]), int(radar.month[d]), int(radar.day[d]), int(radar.hour[d]), int(radar.min[d]), int(radar.sec[d])))

            radar['dates'] = dates

            radar_select = radar[(radar['dates'] >= time_window_i) & (radar['dates'] <= time_window_f)]
            gf_select = gf[(gf['dates'] >= time_window_i) & (gf['dates'] <= time_window_f)]
            print(radar['dates'])
            print(time_window_i, time_window_f)
            print(i)

/Users/andyara/opt/anaconda3/envs/gfz_spacepy/lib/python3.8/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "dtf2d" yielded 1 of "dubious year (Note 6)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),


In [2]:
datetime.datetime.fromtimestamp(radar.ut1_unix[0])

datetime.datetime(2018, 6, 13, 2, 0)

In [3]:
datetime.datetime.fromtimestamp(radar.ut2_unix[0])

datetime.datetime(2018, 6, 13, 2, 1)

In [4]:
radar.head(1)

,year,month,day,hour,min,sec,recno,kindat,kinst,ut1_unix,ut2_unix,azm,elm,hsa,power,systmp,range,pop,dpop,rgate
0,2018,6,13,0,0,30,0,6300,95,1.528848e+09,1.528848e+09,186.800003,81.599998,1.570796,645.1695,95.749359,89.937734,7.055998e+11,5.993838e+10,134.865031


In [5]:
radar_list[i]

'../data/external/Madrigal/2018-2021/EISCATSvalbardISRadarLongyearbyen/MAD6300_2018-06-13_ipy_60@42m.hdf5'

In [6]:
radar.ut1_unix[0]

1528848000.0

In [7]:
import glob
import numpy as np
import pandas as pd
from astropy.time import Time
import cdflib
import datetime
import spacepy.datamodel as dm
from astropy.table import Table

In [8]:
Time(np.mean(radar.ut1_unix, format='unix').datetime

SyntaxError: unexpected EOF while parsing (<ipython-input-8-176a4733ba6b>, line 1)

In [ ]:
Time(radar[['ut1_unix', 'ut2_unix']].mean(axis=1), format='unix').datetime


In [ ]:
i = 64

radar = (Table.read(radar_list[i], path='Data/Table Layout')).to_pandas()
radar_date = datetime.datetime(int(radar.year[0]), int(radar.month[0]), int(radar.day[0])).strftime('%Y-%m-%d')

date, idx_gf, idx_radar = np.intersect1d(gf_list_datetimes, radar_date, return_indices=True)

if len(idx_radar) > 0:
#     print(i,date,idx_gf)

    gf_file = gf_list[idx_gf[0]]
    gf = pd.read_csv(gf_file, sep='\s+', header=0, index_col=False, names=headerlist)
    gf['dates'] = (Time(gf['CDF Epoch'].values, format='cdf_epoch')).datetime

    radar_metadata = (Table.read(radar_list[i], path='Metadata/Experiment Parameters')).to_pandas()

    names = np.array([x.decode() for x in radar_metadata['name']])
    values = np.array([x.decode() for x in radar_metadata['value']])

    radar_metadata = pd.DataFrame({'name': names,
                                   'value': values})
    radar_lat = float(radar_metadata[radar_metadata['name'] == 'instrument latitude']['value'].values[0])
    radar_lon = float(radar_metadata[radar_metadata['name'] == 'instrument longitude']['value'].values[0])

    gf = gf[((gf['Latitude'] <= radar_lat + spatial_window) & (gf['Latitude'] >= radar_lat - spatial_window))]
    gf = gf[((gf['Longitude'] <= radar_lon + spatial_window) & (gf['Longitude'] >= radar_lon - spatial_window))]

    if len(gf) > 0:
#         print('------')
#         print(i, date, len(gf))
#         print('------')

        gf_pass = gf[gf.diff(axis=0)['dates'] > datetime.timedelta(minutes=temporal_window)]
        print(len(gf_pass))
        
        if len(gf_pass) == 1 :
            pass_idx = gf[gf['dates'] == gf_pass['dates'][gf_pass.index.min()]].index[0]
            time_window_i = gf['dates'][gf.index.min()] - datetime.timedelta(minutes=15)
            time_window_f = gf['dates'][gf.index[np.argwhere(gf.index == pass_idx)[0][0] - 1]] + datetime.timedelta(
                    minutes=15)
            print(time_window_i,time_window_f)
            
            radar['dates'] = Time(radar[['ut1_unix', 'ut2_unix']].mean(axis=1), format='unix').datetime
            
            radar_select = radar[(radar['dates'] >= time_window_i) & (radar['dates'] <= time_window_f)]
            print(len(radar_select))

#         if len(gf_pass) == 0:  # only one passage

#             time_window_i = gf['dates'][gf.index.min()] - datetime.timedelta(minutes=temporal_window)
#             time_window_f = gf['dates'][gf.index.max()] + datetime.timedelta(minutes=temporal_window)


#             radar['dates'] = Time(radar[['ut1_unix', 'ut2_unix']].mean(axis=1), format='unix').datetime

#             radar_select = radar[(radar['dates'] >= time_window_i) & (radar['dates'] <= time_window_f)]
#             gf_select = gf[(gf['dates'] >= time_window_i) & (gf['dates'] <= time_window_f)]
#             # print(radar['dates'])
#             # print(time_window_i, time_window_f)
#             # print(i)
#             # exit(0)
#             print(i, len(radar_select))

In [ ]:
(radar['dates']>= time_window_i) & (radar['dates']<= time_window_f)

In [ ]:
radar['dates']

In [ ]:
gf_list[i]

In [ ]:
radar

In [ ]:
len(gf_list)

In [ ]:
radar_date

In [ ]:
idx_gf

In [ ]:
gf_file = gf_list[idx_gf[0]]
gf = pd.read_csv(gf_file, sep='\s+', header=0, index_col=False, names=headerlist)
gf['dates'] = (Time(gf['CDF Epoch'].values, format='cdf_epoch')).datetime

gf['dates']

In [ ]:
gf_list[idx_gf[0]]

In [ ]:
radar_list[i]

In [ ]:
import glob
import numpy as np
import pandas as pd
from astropy.time import Time
import cdflib
import datetime
import spacepy.datamodel as dm
from astropy.table import Table

spatial_window = 5
temporal_window = 15

radar_list = glob.glob('../data/external/Madrigal/2018-2021/*/*.hdf5')
radar_list.sort()
# print(len(radar_list))


gf_list = glob.glob('../data/interim/GRACEFO/KBRNE_relative_v1/dat/*')
gf_list.sort()
# print(len(gf_list))
headerlist = ['CDF Epoch', 'GPS', 'Latitude', 'Longitude', 'Radius', 'Latitude_QD', 'Longitude_QD',
              'MLT GRACE_1_Position', 'GRACE_2_Position', 'Iono_Corr', 'Distance', 'Relative_Hor_TEC', 'Relative_Ne']

gf_list_datetimes = []
for gf_file in gf_list:
    gf = pd.read_csv(gf_file, sep='\s+', header=0, index_col=False, names=headerlist)
    gf_list_datetimes.append((Time(gf['CDF Epoch'].values[-1], format='cdf_epoch').datetime).strftime('%Y-%m-%d'))

for i in iter(range(0, 500)):
    radar = (Table.read(radar_list[i], path='Data/Table Layout')).to_pandas()
    radar_date = datetime.datetime(int(radar.year[0]), int(radar.month[0]), int(radar.day[0])).strftime('%Y-%m-%d')

    date, idx_gf, idx_radar = np.intersect1d(gf_list_datetimes, radar_date, return_indices=True)

    if len(idx_radar) > 0:
        # print(i,date,idx_gf)

        gf_file = gf_list[idx_gf[0]]
        gf = pd.read_csv(gf_file, sep='\s+', header=0, index_col=False, names=headerlist)
        gf['dates'] = (Time(gf['CDF Epoch'].values, format='cdf_epoch')).datetime

        radar_metadata = (Table.read(radar_list[i], path='Metadata/Experiment Parameters')).to_pandas()

        names = np.array([x.decode() for x in radar_metadata['name']])
        values = np.array([x.decode() for x in radar_metadata['value']])

        radar_metadata = pd.DataFrame({'name': names,
                                       'value': values})
        radar_lat = float(radar_metadata[radar_metadata['name'] == 'instrument latitude']['value'].values[0])
        radar_lon = float(radar_metadata[radar_metadata['name'] == 'instrument longitude']['value'].values[0])
        radar['dates'] = Time(radar[['ut1_unix', 'ut2_unix']].mean(axis=1), format='unix').datetime


        gf = gf[((gf['Latitude'] <= radar_lat + spatial_window) & (gf['Latitude'] >= radar_lat - spatial_window))]
        gf = gf[((gf['Longitude'] <= radar_lon + spatial_window) & (gf['Longitude'] >= radar_lon - spatial_window))]

        if len(gf) > 0:
            print(i)
            print(gf['dates'][gf.index.min()])
            print(radar['dates'][radar.index.min()])
            exit(0)

In [9]:
len(radar_list)

7623

In [ ]:
import glob
import numpy as np
import pandas as pd
from astropy.time import Time
import cdflib
import datetime
import spacepy.datamodel as dm
from astropy.table import Table

spatial_window = 5
temporal_window = 15

radar_list = glob.glob('../data/external/Madrigal/2018-2021/*/*.hdf5')
radar_list.sort()
# print(len(radar_list))


gf_list = glob.glob('../data/interim/GRACEFO/KBRNE_relative_v1/dat/*')
gf_list.sort()
# print(len(gf_list))
headerlist = ['CDF Epoch', 'GPS', 'Latitude', 'Longitude', 'Radius', 'Latitude_QD', 'Longitude_QD',
              'MLT GRACE_1_Position', 'GRACE_2_Position', 'Iono_Corr', 'Distance', 'Relative_Hor_TEC', 'Relative_Ne']

gf_list_datetimes = []
for gf_file in gf_list:
    gf = pd.read_csv(gf_file, sep='\s+', header=0, index_col=False, names=headerlist)
    gf_list_datetimes.append((Time(gf['CDF Epoch'].values[-1], format='cdf_epoch').datetime).strftime('%Y-%m-%d'))

for i in iter(range(0, len(radar_list))):
    try:
        radar = (Table.read(radar_list[i], path='Data/Table Layout')).to_pandas()
        radar_date = datetime.datetime(int(radar.year[0]), int(radar.month[0]), int(radar.day[0])).strftime('%Y-%m-%d')

        date, idx_gf, idx_radar = np.intersect1d(gf_list_datetimes, radar_date, return_indices=True)

        if len(idx_radar) > 0:
            # print(i,date,idx_gf)

            gf_file = gf_list[idx_gf[0]]
            gf = pd.read_csv(gf_file, sep='\s+', header=0, index_col=False, names=headerlist)
            gf['dates'] = (Time(gf['CDF Epoch'].values, format='cdf_epoch')).datetime

            radar_metadata = (Table.read(radar_list[i], path='Metadata/Experiment Parameters')).to_pandas()

            names = np.array([x.decode() for x in radar_metadata['name']])
            values = np.array([x.decode() for x in radar_metadata['value']])

            radar_metadata = pd.DataFrame({'name': names,
                                           'value': values})
            radar_lat = float(radar_metadata[radar_metadata['name'] == 'instrument latitude']['value'].values[0])
            radar_lon = float(radar_metadata[radar_metadata['name'] == 'instrument longitude']['value'].values[0])
            radar['dates'] = Time(radar[['ut1_unix', 'ut2_unix']].mean(axis=1), format='unix').datetime


            gf = gf[((gf['Latitude'] <= radar_lat + spatial_window) & (gf['Latitude'] >= radar_lat - spatial_window))]
            gf = gf[((gf['Longitude'] <= radar_lon + spatial_window) & (gf['Longitude'] >= radar_lon - spatial_window))]

            if len(gf) > 0:

                gf_pass = gf[gf.diff(axis=0)['dates'] > datetime.timedelta(minutes=temporal_window)]
                # print(i)
                # print(gf['dates'][gf.index.min()])
                # print(radar['dates'][radar.index.min()])
                # print(len(gf_pass))
                # print('------')

                if len(gf_pass) == 0:  # only one passage

                    time_window_i = gf['dates'][gf.index.min()] - datetime.timedelta(minutes=temporal_window)
                    time_window_f = gf['dates'][gf.index.max()] + datetime.timedelta(minutes=temporal_window)


                    radar['dates'] = Time(radar[['ut1_unix', 'ut2_unix']].mean(axis=1), format='unix').datetime

                    radar_select = radar[(radar['dates'] >= time_window_i) & (radar['dates'] <= time_window_f)]
                    gf_select = gf[(gf['dates'] >= time_window_i) & (gf['dates'] <= time_window_f)]


                    if (len(radar_select) > 0) & (len(gf_select) > 0):
                        print(i, '0 GF passage ', time_window_i, '->', time_window_f)
                        # print('gf : ', len(gf_select))
                        # print('radar : ',len(radar_select))
                        # print('------')

                if len(gf_pass) == 1:  # two passages

                    pass_idx = gf[gf['dates'] == gf_pass['dates'][gf_pass.index.min()]].index[0]

                    # first one

                    # define time window
                    time_window_i = gf['dates'][gf.index.min()] - datetime.timedelta(minutes=15)
                    time_window_f = gf['dates'][gf.index[np.argwhere(gf.index == pass_idx)[0][0] - 1]] + datetime.timedelta(
                        minutes=15)

                    radar_select = radar[(radar['dates'] >= time_window_i) & (radar['dates'] <= time_window_f)]
                    gf_select = gf[(gf['dates'] >= time_window_i) & (gf['dates'] <= time_window_f)]

                    if (len(radar_select) > 0) & (len(gf_select) > 0):
                        print(i, '1 GF passage ', time_window_i, '->', time_window_f)
                        # print('gf : ', len(gf_select))
                        # print('radar : ', len(radar_select))
                        # print('------')

                    # second one

                    # define time window
                    time_window_i = gf['dates'][gf.index[np.argwhere(gf.index == pass_idx)[0][0]]] - datetime.timedelta(
                        minutes=15)
                    time_window_f = gf['dates'][gf.index.max()] + datetime.timedelta(minutes=15)

                    radar_select = radar[(radar['dates'] >= time_window_i) & (radar['dates'] <= time_window_f)]
                    gf_select = gf[(gf['dates'] >= time_window_i) & (gf['dates'] <= time_window_f)]

                    if (len(radar_select) > 0) & (len(gf_select) > 0):
                        print(i, '2 GF passage ', time_window_i, '->', time_window_f)
                        # print('gf : ', len(gf_select))
                        # print('radar : ', len(radar_select))
                        # print('------')
    except:
        print('------')
        print(radar_list[i])
        print(gf_file)
        print('------')
